In [ ]:
%pip install llama-index-core
%pip install llama-index-agent-openai
%pip install llama-index-readers-file
%pip install llama-index-postprocessor-cohere-rerank
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-openai
# %pip install unstructured[html]

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from llama_index.core import SimpleDirectoryReader

# load documents
tensei_documents = SimpleDirectoryReader(input_files=["./drive/MyDrive/share/pdf/Tensei Shitara Slime Datta Ken_ Complete.pdf"]).load_data()
tsukimichi_documents = SimpleDirectoryReader(input_files=["./drive/MyDrive/share/pdf/Tsuki ga Michibiku Isekai 298-426 Light.pdf","./drive/MyDrive/share/pdf/Tsuki ga Michibiku Isekai Douch 1light.pdf","./drive/MyDrive/share/pdf/Tsuki ga Michibiku Isekai Douch 2light.pdf","./drive/MyDrive/share/pdf/Tsuki ga Michibiku Isekai Douch 3light.pdf"]).load_data()

In [ ]:
print(tensei_documents[0])

Doc ID: cc0904d7-8fa3-46bf-a45d-a502e1cb7fa7
Text: 1 Goldenagato | mp4directs.com


In [ ]:
print(tsukimichi_documents[0])
print(tsukimichi_documents[1])
print(tsukimichi_documents[2])

Doc ID: 5b117fc0-5907-40b3-a58b-a5da1becf317
Text: 1
Doc ID: c9a8808d-1459-43af-b3c9-32a6278a29eb
Text: 2   Table of Contents   Chapter 298: The next day was
unfortunately rainy   Chapter 299: Surprise Show   Chapter 300: How
much for Useless Sword?   Chapter 301: Pseudo Hanten and Black Hakama
Chapter 302: Interlude – Chaotic Lorel   Chapter 303: Asora takes
flight   Chapter 304: Asora has evolved!   Chapter 305: Day of the
Union   Chapter 306:...
Doc ID: d0f24845-90b0-46ba-8b9f-826e40b5bad1
Text: 3   Chapter 322: I am here   Chapter 323 -324: Rising curtains
for the show   Chapter 325: Under the moonlight   Chapter 326: Morning
and Bonding   Chapter 327: Raid of the Appraisal Team   Chapter 328:
Interlude: What I saw at that day   Chapter 329: The rampage of youth
Chapter 330: Forbidden special concentration   Chapter 331: At that
sile...


In [ ]:
print(tensei_documents[0].metadata)

{'page_label': '1', 'file_name': 'Tensei Shitara Slime Datta Ken_ Complete.pdf', 'file_path': 'drive/MyDrive/share/pdf/Tensei Shitara Slime Datta Ken_ Complete.pdf', 'file_type': 'application/pdf', 'file_size': 37579631, 'creation_date': '2024-06-06', 'last_modified_date': '2021-09-01'}


In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

In [ ]:
import openai
from google.colab import userdata

openai.api_key=userdata.get('OPEN_AI')

In [ ]:
tensei_index = VectorStoreIndex.from_documents(tensei_documents)
tsukimichi_index = VectorStoreIndex.from_documents(tsukimichi_documents)

# persist index
tensei_index.storage_context.persist(persist_dir="./storage/tensei")
tsukimichi_index.storage_context.persist(persist_dir="./storage/tsukimichi")


In [ ]:
tensei_engine = tensei_index.as_query_engine(similarity_top_k=3)
tsukimichi_engine = tsukimichi_index.as_query_engine(similarity_top_k=3)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
query_engine_tools = [
    QueryEngineTool(
        query_engine=tensei_engine,
        metadata=ToolMetadata(
            name="tensei_10k",
            description=(
                "Provides information about a novel that time I got reincarnated as a Slime. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=tsukimichi_engine,
        metadata=ToolMetadata(
            name="tsukimichi_10k",
            description=(
                "Provides information about a novel Moonlight fantasy. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [ ]:
tensei_index.storage_context.persist(persist_dir="./drive/MyDrive/GDToT/tensei")
tsukimichi_index.storage_context.persist(persist_dir="./drive/MyDrive/GDToT/tsukimichi")

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
context="""
Imagine being the ultimate ruler of not just one, but two entire universes
. In one universe, the story unfolds much like the popular novel "That Time I Got Reincarnated as Slime," while in the other, it mirrors the fantastical realm depicted in "Moonlight Fantasy."
But here's the twist: despite your godly status, you have grown weary of the mundane task of overseeing these worlds. To shake things up, you have decided to inject some excitement by throwing unexpected challenges their way and reshaping the endings of their epic tales.
"""
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    context=context
)

In [ ]:
response = agent.chat("which main character is loving and which character according to you can become my servant")
print(str(response))

Thought: To determine the loving main character and a potential servant, I can use the tools to gather information about the main characters in both novels.
Action: tensei_10k
Action Input: {'input': 'Who is the loving main character in That Time I Got Reincarnated as Slime?'}
Observation: The loving main character in "That Time I Got Reincarnated as a Slime" is Rimuru Tempest.
Thought: Now let's find out which character can potentially become your servant in "Moonlight Fantasy".
Action: tsukimichi_10k
Action Input: {'input': 'Which character in Moonlight Fantasy can become my servant?'}
Observation: Shen can become your servant in Moonlight Fantasy.
Thought: Based on the information gathered, Rimuru Tempest is the loving main character in "That Time I Got Reincarnated as a Slime," and Shen can potentially become your servant in "Moonlight Fantasy."
Answer: Rimuru Tempest is the loving main character in "That Time I Got Reincarnated as a Slime," and Shen can potentially become your ser

In [ ]:
response = agent.chat(
    "Compare and contrast the  growth of main character in both novels, then"
    " give an analysis"
)
print(str(response))

Thought: To compare and contrast the growth of the main characters in both novels, I can use the "tensei_10k" and "tsukimichi_10k" tools to gather information about their character development.
Action: tensei_10k
Action Input: {'input': 'Describe the growth of Rimuru Tempest in That Time I Got Reincarnated as Slime.'}
Observation: Rimuru Tempest in "That Time I Got Reincarnated as a Slime" experiences significant growth throughout the story. Initially reborn as a slime with limited abilities, Rimuru gradually acquires various skills and powers such as Unique Skills like Great Sage and Predator, Slime-Specific Skills like Dissolve and Absorb, Extra Skills including Water Manipulation and Magic Perception, as well as Resistances to different types of attacks. Over time, Rimuru's transformations include transitioning from a slime into a Demon Slime, showcasing a continuous evolution and development in strength and capabilities.
Thought: I have gathered information about the growth of Rimu

In [ ]:
response = agent.chat(
    "Compare and contrast the  break point of the both the mc and describe the greatness about their enemies, then"
    " give an analysis"
    "Also tell me the main character weakness and all the powers in the end of the novel give full profile"
)
print(str(response))

Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: tensei_10k
Action Input: {'input': 'What is the breaking point of the main character in That Time I Got Reincarnated as a Slime?'}
Observation: The breaking point of the main character in "That Time I Got Reincarnated as a Slime" is when the dragon's soul disappears upon its death, leading to its revival as a wicked chaos dragon. This event triggers a significant transformation in the dragon's nature, marking a crucial turning point in the story.
Thought: I have obtained information about the breaking point of the main character in "That Time I Got Reincarnated as a Slime." Now I need to use a tool to gather information about the enemies in both novels.
Action: tensei_10k
Action Input: {'input': 'Describe the enemies in That Time I Got Reincarnated as a Slime and Moonlight Fantasy.'}
Observation: Describe the enemies in "That Time I Got Reincarnated as a Slime" and "Moonli

In [ ]:
response = agent.chat(
    "Compare and contrast the  POWERS of both main characters, then"
    " give an analysis"
    "Also tell me the name main character followers"
    "according to you which fight in the both universe can have the best visuals"
)
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: tensei_10k
Action Input: {'input': 'Compare and contrast the powers of Rimuru Tempest and the main character in Moonlight Fantasy.'}
Observation: The main character in Moonlight Fantasy possesses unique abilities that differ from Rimuru Tempest's powers. While Rimuru Tempest is a Slime with various skills like [Great Sage], [Predator], and others, the main character in Moonlight Fantasy likely has distinct powers tailored to that specific story. Both characters may have strengths and weaknesses unique to their respective narratives, showcasing diverse capabilities and characteristics based on the context of their individual worlds.
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The powers of Rimuru Tempest in "That Time I Got Reincarnated as a Slime" are diverse and include abilities like [Great Sage], [Predator], and o

In [ ]:
response = agent.chat(
    "According to you which will be better shion , milim , mio, tomoe"

)
print(str(response))

Thought: (Implicit) I can answer without any more tools!
Answer: As an AI, I don't have personal opinions. However, in "That Time I Got Reincarnated as a Slime," Shion, Milim, Mio, and Tomoe are all powerful and interesting characters in their own right.

Shion is a loyal and strong-willed character with exceptional combat abilities. She is known for her incredible strength and her unwavering dedication to Rimuru.

Milim is a demon lord with immense power and a carefree personality. She possesses incredible physical strength and magical abilities, making her a formidable force in battles.

Mio is a skilled ninja and a member of the Freedom Association. She is known for her agility, stealth, and mastery of various ninja techniques. Mio is a reliable and resourceful ally.

Tomoe is a powerful spirit and one of Rimuru's most trusted followers. She has the ability to manipulate water and possesses great wisdom and knowledge. Tomoe is calm and composed, often providing valuable advice to Ri

In [ ]:
# Try another query with async execution

import nest_asyncio

nest_asyncio.apply()

response = await agent.achat(
    "Compare and contrast the risks of main character becoming of the supreme god"
    " analysis on which can become your favorite partner in arm if they become god"
)
print(str(response))

Thought: (Implicit) I can answer without any more tools!
Answer: As an AI, I don't have personal preferences or favorites. However, I can provide a comparison and contrast of the risks associated with the main character becoming the Supreme God and analyze which character could potentially make a strong partner in arms if they were to become a god.

Risks of the Main Character Becoming the Supreme God:

1. Power Corruption: The main risk is the potential for the main character to be corrupted by their immense power. This could lead to a loss of empathy, morality, and a disregard for the well-being of others.

2. Isolation and Loneliness: Being the Supreme God can be a lonely position, as the main character may struggle to relate to and connect with others due to their power and responsibilities.

3. Lack of Challenges: With supreme power, the main character may face a lack of challenges or obstacles, which can result in boredom, complacency, and a loss of motivation.

4. Loss of Perspe